In [3]:
from pathlib import Path
import glob
import re
from collections import defaultdict
import logging
from datetime import datetime

In [4]:
from DataObjects.article import Article
from DataObjects.legislation import Legislation, ArticleMapping
from DataObjects.public_consultation import PublicConsultation
from DataObjects.base import Base
from sqlalchemy import create_engine, select, union_all, func, delete, and_, label,cast, Integer
from sqlalchemy.orm import Session, aliased
from configparser import ConfigParser
config = ConfigParser()
config.read("config.ini")
engine = create_engine(config.get('DEFAULT', 'db_file'))

In [5]:
article_mapping_subq = select(Article.number,
                              Article.legislation_id,
                              ArticleMapping.final_legislation_article_no,
                              label(
                                  "final_legislation_article_no_joinNum",
                                  func.coalesce(
                                      func.row_number().over(partition_by=(Article.legislation_id,func.max(ArticleMapping.final_legislation_article_no)),
                                                                           order_by=cast(Article.number,Integer))-1+func.max(ArticleMapping.final_legislation_article_no),
                                      Article.number)
                              )
                            ).where(Article.legislation.has(Legislation.is_public_consultation==1)) \
                            .join(ArticleMapping, and_(Article.legislation_id==ArticleMapping.legislation_id,
                                                     Article.number>=ArticleMapping.public_consultation_article_no
                                                     )) \
                            .group_by(Article.number,Article.legislation_id) \
                            .subquery()

final_legislations_subq = select(Legislation).where(Legislation.is_public_consultation==0).subquery()
final_legislations_articles = aliased(Article)
stmt = select(Legislation.id,
              Article.number,
              Article.text,
              article_mapping_subq.c.final_legislation_article_no_joinNum,
              final_legislations_articles.text,
              final_legislations_articles.number).where(Article.legislation.has(Legislation.is_public_consultation==1)) \
                        .join(article_mapping_subq, and_(Article.number==article_mapping_subq.c.number,
                                                         Article.legislation_id==article_mapping_subq.c.legislation_id)
                                                         ) \
                        .join(Legislation) \
                        .join(final_legislations_articles,and_(final_legislations_articles.legislation_id==Legislation.final_legislation_id,
                                                               article_mapping_subq.c.final_legislation_article_no_joinNum==final_legislations_articles.number))

with Session(engine) as sess:
    rows = sess.execute(stmt)
    i=0
    for row in rows:
        print(row[2][:50],'|',row[4][:50])
    

 Σκοπός του παρόντος νόμου είναι ο εκσυγχρονισμός  | 
Σκοπός του παρόντος νόµου είναι:
α) Ο εκσυγχρονισ
 Αντικείμενο του παρόντος είναι η αντικατάσταση δι | 
Αντικείµενο του παρόντος είναι:
α) Η αντικατάστασ
 1. Οι Φαρμακευτικοί Σύλλογοι είναι νομικά πρόσωπα | 
1. Οι Φαρµακευτικοί Σύλλογοι είναι νοµικά πρόσωπα
 Σκοποί του Φαρμακευτικού Συλλόγου είναι:α) Η προα | 
Σκοποί του Φαρµακευτικού Συλλόγου είναι:
α) Η προ
 Οι Φαρμακευτικοί Σύλλογοι διατηρούν την ίδια τοπι | 
Οι Φαρµακευτικοί Σύλλογοι διατηρούν την ίδια τοπι
 1. Μέλη του Φαρμακευτικού Συλλόγου είναι υποχρεωτ | 
1. Μέλη του Φαρµακευτικού Συλλόγου είναι υποχρεω-
 1. Ο φαρμακοποιός, στον οποίο έχει χορηγηθεί άδει | 
1. Ο φαρµακοποιός, στον οποίο έχει χορηγηθεί άδει
 1. Στα γραφεία εκάστου Φαρμακευτικού Συλλόγου τηρ | 
1. Στα γραφεία εκάστου Φαρµακευτικού Συλλόγου τη-
 Ο Φαρμακευτικός Σύλλογος διοικείται από τη Γενική | 
Ο Φαρµακευτικός Σύλλογος διοικείται από τη Γενική
 1. Η σφραγίδα των Φαρμακευτικών Συλλόγων είναι κυ | 
1. Η σφραγ